## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [26]:
import os


# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [27]:
import keras
from keras.models import Model
from keras.utils import to_categorical
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau,EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
import os

In [28]:
batch_size = 64 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練整個資料集共 30個循環

In [29]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [30]:
# 建立 ResNet 模型
model = resnet(input_shape=(32,32,3)) 
model.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_93 (Conv2D)              (None, 32, 32, 16)   448         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_84 (BatchNo (None, 32, 32, 16)   64          conv2d_93[0][0]                  
__________________________________________________________________________________________________
activation_84 (Activation)      (None, 32, 32, 16)   0           batch_normalization_84[0][0]     
_______________________________________________________________________________________

In [31]:

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

augment_generator = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

reduce_lr = ReduceLROnPlateau(factor=0.0009, 
                              min_lr=1e-12, 
                              monitor='val_loss', 
                              patience=10, 
                              verbose=1)

earlystop = EarlyStopping(monitor="val_acc", 
                          patience=10, 
                          verbose=1
                          )

history = model.fit_generator(augment_generator.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=int(len(x_train)/batch_size), # 一個 epochs 要執行幾次 update，通常是資料量除以 batch size
                    epochs=epochs,
                    verbose=1,
                    callbacks=[reduce_lr,earlystop],          
                    validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/30
781/781 [==============================] - 818s 1s/step - loss: 1.8679 - accuracy: 0.4957 - val_loss: 2.3423 - val_accuracy: 0.4320
Epoch 2/30
781/781 [==============================] - 902s 1s/step - loss: 1.3966 - accuracy: 0.6262 - val_loss: 1.5311 - val_accuracy: 0.5810
Epoch 3/30
781/781 [==============================] - 959s 1s/step - loss: 1.2107 - accuracy: 0.6824 - val_loss: 1.7460 - val_accuracy: 0.5765
Epoch 4/30
781/781 [==============================] - 954s 1s/step - loss: 1.0877 - accuracy: 0.7202 - val_loss: 1.5490 - val_accuracy: 0.6028
Epoch 5/30
781/781 [==============================] - 945s 1s/step - loss: 0.9944 - accuracy: 0.7493 - val_loss: 1.4603 - val_accuracy: 0.6388
Epoch 6/30
781/781 [==============================] - 961s 1s/step - loss: 0.9419 - accuracy: 0.7674 - val_loss: 1.0800 - val_accuracy: 0.7150
Epoch 7/30
781/781 [==============================] - 972s 1s/step - loss: 0.8996 - accuracy: 0.7797 - val_loss: 0.9214 - val_accuracy: 0.7727

781/781 [==============================] - 944s 1s/step - loss: 0.6302 - accuracy: 0.8598 - val_loss: 0.8215 - val_accuracy: 0.7993
Epoch 23/30
781/781 [==============================] - 943s 1s/step - loss: 0.6163 - accuracy: 0.8638 - val_loss: 0.8459 - val_accuracy: 0.8085
Epoch 24/30
781/781 [==============================] - 939s 1s/step - loss: 0.6167 - accuracy: 0.8630 - val_loss: 0.7715 - val_accuracy: 0.8210
Epoch 25/30
781/781 [==============================] - 949s 1s/step - loss: 0.6071 - accuracy: 0.8665 - val_loss: 0.7338 - val_accuracy: 0.8284
Epoch 26/30
781/781 [==============================] - 936s 1s/step - loss: 0.6001 - accuracy: 0.8683 - val_loss: 1.2216 - val_accuracy: 0.7073
Epoch 27/30
781/781 [==============================] - 945s 1s/step - loss: 0.5916 - accuracy: 0.8727 - val_loss: 0.9365 - val_accuracy: 0.7742
Epoch 28/30
781/781 [==============================] - 918s 1s/step - loss: 0.5875 - accuracy: 0.8736 - val_loss: 0.8750 - val_accuracy: 0.8016
Epoc